In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.output {font-size:10pt;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
</style>
"""))

- class를 이용하여 출력하기

In [1]:
class Member:
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
    
    def __str__(self):
        return "{:>5}\t{:3}\t{:15}\t{:15}\t{:3}\t{}".format(
            '*'*self.grade, self.name, self.phone, self.email, self.age, self.etc)
    
    def to_dict(self):
        return {"name":self.name, "phone":self.phone, "email":self.email,
               "age":self.age, "grade":self.grade, "etc":self.etc}
    
def to_member(*row): # 튜플 데이터를 매개변수로 받아 Member형 객체로 변환
    return Member(row[0], row[1], row[2], row[3], row[4], row[5])

In [2]:
# 1.입력
def insert_member_info():
    cursor = conn.cursor()
    
    name = input('이름: ')
    phone = input('전화번호: ')
    email = input('이메일: ')
    try:
        age = int(input('나이: '))
    except ValueError as e:
        print('올바르지 않은 값을 입력시 0으로 초기화합니다.')
        age = 0
    
    try:
        grade = int(input('고객등급(1~5): '))
        if grade < 0:
            grade = 0
        elif grade > 5:
            grade = 5 
    except ValueError as e:
        print('올바르지 않은 값을 입력시 1로 초기화합니다.')
        grade = 1 
    etc = input('기타 정보: ')
    
    member = Member(name, phone, email, age, grade, etc)
    cursor.execute("""
        INSERT INTO MEMBERS VALUES 
        (:name, :phone, :email, :age, :grade, :etc)
    """, member.to_dict())
    
    conn.commit()
    cursor.close()

In [3]:
# 2. 전체 조회
def print_members():
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MEMBERS")
    members = cursor.fetchall()
    
    print('='*70)
    print("{:^70}".format("고객 정보"))
    print('-'*70)
    print("{}\t{}\t{}\t\t{}\t\t{}\t{}".format(
            "GRADE","이름","전화","메일","나이","기타"))
    print('='*70)    
    for member in members:
        print(to_member(*member))
    
    if len(members) == 0:
        print('저장된 회원이 없습니다.')
    print('='*70)
    
    cursor.close()

In [4]:
# 3. 이름 검색 
def search_member():
    cursor = conn.cursor()
    
    name = input("이름: ")
    cursor.execute("SELECT * FROM MEMBERS WHERE NAME = ?", (name,))
    members = cursor.fetchall()
    for member in members:
        print(to_member(*member))
    if len(members) == 0:
        print('해당 이름의 회원이 존재하지 않습니다.')

In [5]:
# 4. 메일로 삭제하기
def delete_member():
    cursor = conn.cursor()
    
    email = input('이메일: ')
    cursor.execute("SELECT * FROM MEMBERS WHERE EMAIL=:email", {'email':email})
    members = cursor.fetchall()
    
    if len(members) != 0:
        cursor.execute('DELETE FROM MEMBERS WHERE EMAIL=:email', {'email':email})
        conn.commit()
        print('요청하신 정보를 삭제하였습니다.')
    else: 
        print('요청하신 메일이 데이터에 존재하지 않습니다.')
    cursor.close()

In [10]:
# 5. 데이터베이스에서 조회한 정보를 CSV파일로 저장 
def export_csv_member():
    import csv
    
    filename = input('파일명(csv확장자 포함): ')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM MEMBERS')
    members = cursor.fetchall()
    colnames = [row[0] for row in cursor.description]
    # colnames = [row[0].lower() for row in cursor.description]
    cursor.close()
    
    with open(filename, 'w', newline='', encoding='utf8') as file:
        w = csv.writer(file)
        w.writerow(colnames)
        w.writerows(members)

In [7]:
# 메인 함수
def main():
    while True:
        print("1:입력", "2:전체 출력", "3: 이름 검색", 
             "4:메일 삭제", "5: 내보내기(CSV)", "0:종료",
             sep = " | ", end = "")
        menu = int(input("메뉴 선택: "))
    
        if menu == 1:
            insert_member_info()
        elif menu == 2:
            print_members()
        elif menu == 3:
            search_member()
        elif menu == 4:
            delete_member()
        elif menu == 5:
            export_csv_member()
        elif menu == 0:
            conn.close()
            break;

In [12]:
# 실행 
if __name__ == '__main__':
    import sqlite3
    global conn
    conn = sqlite3.connect('data/data.db')
    main()

1:입력 | 2:전체 출력 | 3: 이름 검색 | 4:메일 삭제 | 5: 내보내기(CSV) | 0:종료메뉴 선택: 2
                                고객 정보                                 
----------------------------------------------------------------------
GRADE	이름	전화		메일		나이	기타
    *	Song	010            	song@si.com    	  3	friend
   **	Shin	011            	shin@hj.com    	 30	friend
1:입력 | 2:전체 출력 | 3: 이름 검색 | 4:메일 삭제 | 5: 내보내기(CSV) | 0:종료메뉴 선택: 1
이름: Park
전화번호: 012
이메일: park@sa.com
나이: 30
고객등급(1~5): 3
기타 정보: friend
1:입력 | 2:전체 출력 | 3: 이름 검색 | 4:메일 삭제 | 5: 내보내기(CSV) | 0:종료메뉴 선택: 2
                                고객 정보                                 
----------------------------------------------------------------------
GRADE	이름	전화		메일		나이	기타
    *	Song	010            	song@si.com    	  3	friend
   **	Shin	011            	shin@hj.com    	 30	friend
  ***	Park	012            	park@sa.com    	 30	friend
1:입력 | 2:전체 출력 | 3: 이름 검색 | 4:메일 삭제 | 5: 내보내기(CSV) | 0:종료메뉴 선택: 3
이름: park
해당 이름의 회원이 존재하지 않습니다.
1:입력 | 2:전체 출력 | 3: 이름 검색 | 4:메일 삭제

**class를 이용하지 않고 출력하기**

In [ ]:
# 1. 입력 
def insert_member_info():
    cursor = conn.cursor()
    
    name = input('이름: ')
    phone = input('전화번호: ')
    email = input('이메일: ')
    try:
        age = int(input('나이: '))
    except ValueError as e:
        print('유효하지 않은 값 입력 시, 값은 0으로 초기화됩니다.')
        age = 0
    
    try:
        grade = int(input('고객등급(1~5): '))
        if grade < 0:
            grade = 0
        elif grade > 5:
            grade = 5
    except ValueError as e:
        print('유효하지 않은 값 입력시, 값은 1로 초기화됩니다.')
        grade = 1
    
    etc = input('기타정보: ')
    
    cursor execute("""
        INSERT INTO member VALUES 
            (:name, :phone, :email, :age, :grade, :etc)""", 
        {'name':name, 'phone':phone, 'email':email,
         'age':age, 'grade':grade, 'etc',:etc})
    conn.commit
    cursor.close()

In [ ]:
# 2. 전체 출력 
def print_members():
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MEMBER")
    members = cursor.fetchall()
    
    print('='*70)
    print("{:^70}".format('고객 정보'))
    print('-'*70)
    print("{}\t{}\t{}\t\t{}\t\t{}\t{}".format('GRADE', '이름', '전화', '메일', '나이', '기타'))
    print('='*70)
    
    for member in members:
        print("{}\t{}\t{}\t\t{}\t\t{}\t{}".format('*'*int(member[4]), member[0], member[1], 
                                                  member[2], member[3], member[5]))
    if len(members) == 0:
        print('회원 정보가 없습니다.')
    print('='*70)
    
    cursur.close()        

In [ ]:
# 3. 이름 찾기
def search_member():
    cursur = conn.cursor()
    name = input('이름 검색: ')
    cursor.execute("SELECT * FROM MEMBER WHERE NAME=?", (name,))
    members = cursor.fetchall()
    
    for member in members:
        print("{}\t{}\t{}\t\t{}\t\t{}\t{}".format('*'*int(member[4]), member[0], member[1], 
                                                  member[2], member[3], member[5]))
    if len(members) == 0:
        print('검색하신 이름을 찾을 수 없습니다.')

In [ ]:
# 4. 메일 삭제 
def delete_member():
    cursor = conn.cursor()
    email = input("삭제할 회원 이메일: ")
    cursor.execute('SELECT * FROM MEMBER WHERE EMAIL=:email', {'email':email})
    members = cursur.fetchall()
    
    if len(members) != 0:
        cursor.execute('DELETE FROM MEMBER WHERE EMAIL=:email',{'email':email})
        conn.commit()
        print('요청하신 회원을 삭제하였습니다.')
    else:
        print('검색하신 이메일을 찾을 수 없습니다.')
    
    cursor.close()

In [ ]:
# 5. 내보내기(csv)
def export_csv_member():
    file_name = input('파일명(csv확장자 포함): ')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MEMBER")
    members = cursor.fetchall()
    colnames = [row[0] for row in cursor.description]
    # colnames = [row[0].lower() for row in cursor.description]
    corsur.close()
    
    import csv
    with open(file_name, 'w', newline='', encoding='UTF8') as file:
        w = csv.writer(file)
        w.writerow(colnames)
        w.writerows(members)

In [3]:
def main():
    while True:
        print("1:입력", "2:전체출력", "3:이름찾기", "4:메일삭제", 
              "5:내보내기(CSV)", "0:종료", sep="|", end="")
        menu = int(input('메뉴 선택: '))
        
        if menu == 1:
            insert_member_info()
        elif menu == 2:
            print_members()
        elif menu == 3:
            search_member()
        elif menu == 4:
            delete_member()
        elif menu == 5:
            export_csv_member()
        elif menu == 0:
            conn.close()
            break;

In [ ]:
if __name__=='__main__':
    import sqlite3
    global conn
    conn = sqlite3.connect('data.db')
    main()

1:입력|2:전체출력|3:이름찾기|4:메일삭제|5:내보내기(CSV)|0:종료